In [19]:
import cdm_pytorch as cdm
import csv

In [20]:
# val_loss, tr_loss, gv, train_ds, val_ds, model, opt = cdm.default_run(dataset='syn_nature_triplet', epochs=2000, wd=0.003)

In [21]:
# we want to run this model on similarity data, so we need to change the model a bit.
# before we do that, we need to understand the code a little better.
# we should go step by step in the default_run function and understand what each line does.
# train_ds, val_ds, dm, num_items, ism = load_data(dataset,dd,dm,ism,extra_name, target_name,seed=seed)
# train_dl, val_dl = get_data(train_ds, val_ds, batch_size=batch_size)
# model, opt = get_model(Model, num_items, embedata_dicting_dim, ism, lr, wd=wd, seed=seed, **kwargs)
# tr_loss, val_loss, gv = fit(epochs, model, opt, train_dl, val_dl)

In [22]:
# the best way to understand the code is to try to replicate it for the similarity dataset
# I think the code can handle datasets of different sizes, so we should be able to use the same code for the food similarity dataset
# at the same time, we should also try the face similarity dataset

In [23]:
# understanding load_data
# load_data(dataset=None, data_dict=None, dm=None, ism=True, extra_name='choice_set_lengths',
#               target_name='slot_chosen', seed=None):
#     if dataset is not None:
#         dd, dm, _, ism = cu.read_and_id_data(dataset)
#     data_size, num_items = len(data_dict[target_name]), len(dm)

#     whole_ds = list(map(t.tensor, [data_dict['context_ids'], dd[extra_name], dd[target_name]]))
#     train_ds, val_ds = random_split(whole_ds, [int(.8*data_size), data_size-int(.8*data_size)], seed=seed)

#     return train_ds, val_ds, dm, num_items, ism

# need to understand what is dd, dm, ism

#ism = ismultiset is a flag for whether the multiple set sizes are used (so padding can be used if necessary)

# def read_and_id_data(dataset, filename=None, pre_proc=True, context_size=None,
	# 				cluster=False, k=50):
	# # Wrapper for read_data and feature_id that makes this process easier
	# if filename is None:
	# 	filename = '../data/raw/'+dataset+'_data_final.csv'
	# data_dict = read_data(filename,pre_proc)
	# dh = dataset_headers(dataset)
	# data_map, context_size, ismultiset = feature_id(data_dict,dh,context_size,
	# 												cluster,k)

	# return data_dict, data_map, context_size, ismultiset

# def get_data(train_ds, val_ds, batch_size=None):
#     # Note: can change val_bs to 2* batch_size if ever becomes a problem
#     if batch_size is not None:
#         tr_bs, val_bs = (batch_size, len(val_ds[0]))
#     else: 
#         tr_bs, val_bs = (len(train_ds[0]), len(val_ds[0]))

#     train_dl = DataLoader(train_ds, batch_size=tr_bs, shuffle=batch_size is not None)
#     val_dl = DataLoader(val_ds, batch_size=val_bs)
#     return train_dl, val_dl

# get_data is fairly straightforward. It takes the train_ds and val_ds and creates DataLoader objects from them.



In [24]:
dataset = 'syn_nature_triplet'
features = ['item']
filename = '../data/raw/'+dataset+'_data_final.csv'
data_dict = {}
with open(filename, mode='r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        for k, v in row.items():
            if v != '':
                try:
                    data_dict.setdefault(k,[]).append(int(v))
                except:
                    data_dict.setdefault(k,[]).append(v)
            else:
                data_dict.setdefault(k,[]).append(None)


In [25]:
for key in data_dict:
    print(key, data_dict[key][:10])

 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
turker_id [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
item1 ['n255068.jpg', 'bost101.jpg', 'bost101.jpg', 'n255068.jpg', 'n255068.jpg', 'n255068.jpg', 'n255068.jpg', 'n255068.jpg', 'n255068.jpg', 'land645.jpg']
item2 ['bost101.jpg', 'land684.jpg', 'land684.jpg', 'land684.jpg', 'bost101.jpg', 'n219007.jpg', 'n219007.jpg', 'n219007.jpg', 'land684.jpg', 'land661.jpg']
item3 ['land684.jpg', 'n255068.jpg', 'n255068.jpg', 'bost101.jpg', 'land684.jpg', 'land661.jpg', 'land661.jpg', 'land661.jpg', 'bost101.jpg', 'n255068.jpg']
slot_chosen [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [26]:
import sys
sys.path.insert(0, '../utils/')
import choice_utils as cu
import re


In [27]:
matches = [re.match(features[0]+r'(\d+)', k) for k in data_dict.keys()]
print(matches)
context_size = max([int(m.group(1)) for m in matches if m])
print(context_size)

[None, None, <re.Match object; span=(0, 5), match='item1'>, <re.Match object; span=(0, 5), match='item2'>, <re.Match object; span=(0, 5), match='item3'>, None]
3


In [28]:
temp_list = []
for i in range(len(features)):
    temp_list.append([])
    for j in range(1,context_size+1):
        temp_list[-1] += data_dict[features[i]+str(j)]

In [29]:
temp_list

[['n255068.jpg',
  'bost101.jpg',
  'bost101.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'land645.jpg',
  'n255068.jpg',
  'land645.jpg',
  'land645.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'land645.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'land645.jpg',
  'n251048.jpg',
  'n255068.jpg',
  'n251048.jpg',
  'n255068.jpg',
  'n251048.jpg',
  'n251048.jpg',
  'n251048.jpg',
  'n213066.jpg',
  'n213066.jpg',
  'n213066.jpg',
  'n213066.jpg',
  'n251048.jpg',
  'n213066.jpg',
  'land902.jpg',
  'n255068.jpg',
  'land902.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n295018.jpg',
  'land902.jpg',
  'n295018.jpg',
  'n255068.jpg',
  'n295018.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'land902.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg',
  'n255068.jpg

In [30]:
data_map = cu.feature_map(temp_list)

In [31]:
data_map

{('n255068.jpg',): 0,
 ('bost101.jpg',): 1,
 ('land645.jpg',): 2,
 ('n251048.jpg',): 3,
 ('n213066.jpg',): 4,
 ('land902.jpg',): 5,
 ('n295018.jpg',): 6,
 ('land688.jpg',): 7,
 ('cdmc292.jpg',): 8,
 ('land661.jpg',): 9,
 ('cdmc318.jpg',): 10,
 ('land812.jpg',): 11,
 ('land656.jpg',): 12,
 ('cdmc317.jpg',): 13,
 ('n219015.jpg',): 14,
 ('land369.jpg',): 15,
 ('land480.jpg',): 16,
 ('land372.jpg',): 17,
 ('cdmc281.jpg',): 18,
 ('n219008.jpg',): 19,
 ('cdmc290.jpg',): 20,
 ('land665.jpg',): 21,
 ('n203004.jpg',): 22,
 ('cdmc306.jpg',): 23,
 ('n213093.jpg',): 24,
 ('land924.jpg',): 25,
 ('land639.jpg',): 26,
 ('cdmc315.jpg',): 27,
 ('n213092.jpg',): 28,
 ('land370.jpg',): 29,
 ('land623.jpg',): 30,
 ('bost103.jpg',): 31,
 ('land954.jpg',): 32,
 ('land374.jpg',): 33,
 ('land339.jpg',): 34,
 ('land334.jpg',): 35,
 ('n213097.jpg',): 36,
 ('land616.jpg',): 37,
 ('n295042.jpg',): 38,
 ('n344019.jpg',): 39,
 ('cdmc277.jpg',): 40,
 ('bost98.jpg',): 41,
 ('n213080.jpg',): 42,
 ('cdmc101.jpg',): 43,

In [32]:
data_map, context_size, ismultiset = cu.feature_id(data_dict, features, context_size=None, cluster=False, k=50)

In [33]:
data_dict.keys()

dict_keys(['', 'turker_id', 'item1', 'item2', 'item3', 'slot_chosen', 'context_ids', 'choice_id', 'context_ids_wo_choice', 'choice_set_lengths'])

In [34]:
data_dict['context_ids']

[[0, 1, 115],
 [1, 115, 0],
 [1, 115, 0],
 [0, 115, 1],
 [0, 1, 115],
 [0, 111, 9],
 [0, 111, 9],
 [0, 111, 9],
 [0, 115, 1],
 [2, 9, 0],
 [0, 111, 9],
 [2, 9, 0],
 [2, 9, 0],
 [0, 111, 9],
 [0, 100, 9],
 [2, 9, 0],
 [0, 100, 9],
 [0, 100, 9],
 [2, 0, 9],
 [3, 0, 9],
 [0, 100, 9],
 [3, 0, 9],
 [0, 100, 9],
 [3, 0, 9],
 [3, 0, 9],
 [3, 0, 9],
 [4, 0, 9],
 [4, 0, 9],
 [4, 0, 9],
 [4, 0, 9],
 [3, 0, 9],
 [4, 0, 9],
 [5, 9, 0],
 [0, 9, 5],
 [5, 0, 9],
 [0, 6, 9],
 [0, 9, 5],
 [6, 0, 5],
 [5, 9, 0],
 [6, 0, 5],
 [0, 6, 9],
 [6, 0, 5],
 [0, 6, 9],
 [0, 6, 9],
 [5, 9, 0],
 [0, 102, 9],
 [0, 102, 5],
 [0, 6, 9],
 [0, 102, 9],
 [0, 102, 5],
 [0, 102, 9],
 [0, 102, 5],
 [0, 102, 5],
 [0, 105, 9],
 [0, 102, 5],
 [0, 105, 5],
 [0, 105, 9],
 [0, 105, 5],
 [0, 105, 5],
 [0, 105, 9],
 [0, 102, 5],
 [0, 105, 9],
 [7, 9, 0],
 [5, 0, 9],
 [0, 7, 9],
 [0, 105, 9],
 [7, 0, 5],
 [5, 7, 0],
 [7, 0, 5],
 [0, 7, 9],
 [7, 9, 0],
 [0, 7, 9],
 [0, 104, 5],
 [7, 0, 5],
 [0, 104, 5],
 [0, 104, 9],
 [0, 104, 9],
 [

In [35]:
data_dict['choice_id']

[115,
 0,
 0,
 1,
 115,
 9,
 9,
 9,
 1,
 0,
 9,
 0,
 0,
 9,
 9,
 0,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 0,
 5,
 9,
 9,
 5,
 5,
 0,
 5,
 9,
 5,
 9,
 9,
 0,
 9,
 5,
 9,
 9,
 5,
 9,
 5,
 5,
 9,
 5,
 5,
 9,
 5,
 5,
 9,
 5,
 9,
 0,
 9,
 9,
 9,
 5,
 0,
 5,
 9,
 0,
 9,
 5,
 5,
 5,
 9,
 9,
 9,
 9,
 5,
 9,
 0,
 5,
 5,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 9,
 0,
 9,
 5,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 11,
 8,
 11,
 11,
 0,
 0,
 0,
 9,
 0,
 5,
 5,
 0,
 5,
 0,
 0,
 9,
 0,
 0,
 69,
 0,
 0,
 5,
 0,
 10,
 5,
 0,
 0,
 5,
 0,
 0,
 13,
 0,
 0,
 0,
 15,
 9,
 15,
 0,
 9,
 0,
 0,
 9,
 0,
 0,
 0,
 16,
 16,
 0,
 0,
 69,
 0,
 0,
 17,
 0,
 10,
 0,
 0,
 0,
 10,
 0,
 17,
 0,
 0,
 69,
 9,
 0,
 0,
 0,
 10,
 18,
 0,
 0,
 18,
 0,
 69,
 0,
 10,
 0,
 0,
 0,
 18,
 0,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 5,
 9,
 0,
 0,
 9,
 0,
 0,
 9,
 25,
 5,
 0,
 9,
 25,
 0,
 25,
 9,
 0,
 0,
 25,
 0,
 0,
 0,
 0,
 9

In [36]:
data_dict['context_ids_wo_choice']

[[0, 1],
 [1, 115],
 [1, 115],
 [0, 115],
 [0, 1],
 [0, 111],
 [0, 111],
 [0, 111],
 [0, 115],
 [2, 9],
 [0, 111],
 [2, 9],
 [2, 9],
 [0, 111],
 [0, 100],
 [2, 9],
 [0, 100],
 [0, 100],
 [2, 0],
 [3, 0],
 [0, 100],
 [3, 0],
 [0, 100],
 [3, 0],
 [3, 0],
 [3, 0],
 [4, 0],
 [4, 0],
 [4, 0],
 [4, 0],
 [3, 0],
 [4, 0],
 [5, 9],
 [0, 9],
 [5, 0],
 [0, 6],
 [0, 9],
 [6, 0],
 [5, 9],
 [6, 0],
 [0, 6],
 [6, 0],
 [0, 6],
 [0, 6],
 [5, 9],
 [0, 102],
 [0, 102],
 [0, 6],
 [0, 102],
 [0, 102],
 [0, 102],
 [0, 102],
 [0, 102],
 [0, 105],
 [0, 102],
 [0, 105],
 [0, 105],
 [0, 105],
 [0, 105],
 [0, 105],
 [0, 102],
 [0, 105],
 [7, 9],
 [5, 0],
 [0, 7],
 [0, 105],
 [7, 0],
 [5, 7],
 [7, 0],
 [0, 7],
 [7, 9],
 [0, 7],
 [0, 104],
 [7, 0],
 [0, 104],
 [0, 104],
 [0, 104],
 [0, 7],
 [0, 104],
 [0, 104],
 [0, 104],
 [8, 9],
 [0, 104],
 [0, 104],
 [0, 104],
 [8, 9],
 [8, 9],
 [5, 8],
 [5, 8],
 [5, 8],
 [5, 8],
 [5, 8],
 [0, 8],
 [0, 10],
 [9, 10],
 [0, 10],
 [10, 0],
 [10, 5],
 [10, 5],
 [9, 10],
 [10, 5],
 

In [37]:
data_dict['choice_set_lengths']

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [38]:
import torch as t
from cdm_pytorch import random_split

In [39]:
data_size, num_items = len(data_dict['slot_chosen']), len(data_map)
whole_ds = list(map(t.tensor, [data_dict['context_ids'], data_dict['choice_set_lengths'], data_dict['slot_chosen']]))
train_ds, val_ds = random_split(whole_ds, [int(.8*data_size), data_size-int(.8*data_size)])

In [42]:
whole_ds

[tensor([[  0,   1, 115],
         [  1, 115,   0],
         [  1, 115,   0],
         ...,
         [ 73,  89,  12],
         [ 97, 110,  65],
         [ 20,  64, 117]]),
 tensor([3, 3, 3,  ..., 3, 3, 3]),
 tensor([2, 2, 2,  ..., 2, 2, 2])]

In [41]:
# def get_data(train_ds, val_ds, batch_size=None):
#     # Note: can change val_bs to 2* batch_size if ever becomes a problem
#     if batch_size is not None:
#         tr_bs, val_bs = (batch_size, len(val_ds[0]))
#     else: 
#         tr_bs, val_bs = (len(train_ds[0]), len(val_ds[0]))

#     train_dl = DataLoader(train_ds, batch_size=tr_bs, shuffle=batch_size is not None)
#     val_dl = DataLoader(val_ds, batch_size=val_bs)
#     return train_dl, val_dl

train_dl, val_dl = cdm.get_data(train_ds, val_ds, batch_size=32)